In [1]:
import pickle,os,pandas

In [2]:
# Path where all the predicted topics would be present
#here = os.path.abspath("C:\\Users\\Beast\\Desktop\\Project_Run")
here = os.path.abspath("C:\\Users\\Beast\\Desktop\\Project_Run\\clean")

In [3]:
picklepath = os.path.abspath("C:\\Users\\Beast\\Desktop\\Ted_confx\\indexfiles\\clean\\topic_bags_20.pkl")
p1 = open(picklepath,"rb")
topic_bags = pickle.load(p1)
p1.close()

In [4]:
# Get the files for topics
topic_for_file = pandas.read_csv("C:\\Users\\Beast\\Desktop\\Ted_confx\\topics.csv",encoding = "ISO-8859-1")

In [5]:
# Create a dictionary with the list of actual topics for every file

actual_topic_dict = {}

for file in topic_for_file['Filename']:
    rowf = topic_for_file.loc[topic_for_file['Filename'] == file]  #get the row ie filename row
    rowfp = [x.upper() for x in list(rowf.iloc[0][6:]) if str(x) != 'nan'] #get the list of topics for the file
    actual_topic_dict[file] = rowfp                                       #add to the dictionary
    

In [6]:
# Create the list of files that need to be processed
file_list = []
for file in os.listdir(here):
    if file.endswith(".txt"):
        file_list.append(file)

In [7]:
# Predict 15
predict_file_dict = {}
for file in file_list:
    fptest = os.path.join(here,file)
    file_t = open(fptest,"r")
    test_file = file_t.read()
    topic_dict = {}
    for topic in topic_bags:
        match_words = set(test_file) & set(topic_bags[topic])
        if len(match_words) > 0:
            topic_dict[topic] = len(match_words)
    sorted_topics = sorted(topic_dict.items(),key=lambda x: x[1],reverse=True)
    predict_file_dict[file] = [list(ele)[0] for ele in sorted_topics]
    file_t.close()

In [8]:
final_eval = {}
sum_mp,accuracy = 0,0
for file in topic_for_file['Filename']:
    match = set(actual_topic_dict[file]) & set(predict_file_dict[file])
    match_percent = (len(match)*100)/len(actual_topic_dict[file])
    final_eval[file] = [actual_topic_dict[file],predict_file_dict[file],match_percent]
    sum_mp += match_percent
accuracy = sum_mp/len(topic_for_file)

In [9]:
accuracy #final_eval

8.157843147153484

In [10]:
# Save the final evaluation i.e after prediction
picklepath = os.path.abspath("C:\\Users\\Beast\\Desktop\\Project_Run\\test_final_eval_topic_bags_20.pkl")
p1 = open(picklepath,"wb")
pickle.dump(final_eval, p1, pickle.HIGHEST_PROTOCOL)
p1.close()